In [1]:
import cv2
import os
import numpy as np 
import tensorflow as tf 

In [2]:
# Fonction pour charger les images du dataset et les mettre en vecteurs numpy
def load_dataset(directory, label):
    images = []
    target = []
    counter = 0
    # Itérations sur les dossiers et fichiers
    for root, dirs, files in os.walk(directory):
        for name in files:
            counter += 1
            # Ajout des images à la liste d'images et de la target
            try:
                img = cv2.imread(os.path.join(root, name))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (48, 48))

                images.append(img)
                target.append(label)
            except:
                continue

    images = np.array(images) /255.0
    images = np.reshape(images, (-1, 48, 48, 1))
    target = np.array(target)

    return (images, target)

In [3]:
# Génération des différentes listes de train
angry_train = load_dataset("data/train/angry", 0)
disgust_train = load_dataset("data/train/disgust", 1)
fear_train = load_dataset("data/train/fear", 2)
happy_train = load_dataset("data/train/happy", 3)
neutral_train = load_dataset("data/train/neutral", 4)
sad_train = load_dataset("data/train/sad", 5)
surprise_train = load_dataset("data/train/surprise", 6)

In [4]:
# Génération des différentes listes de test
angry_test = load_dataset("data/test/angry", 0)
disgust_test = load_dataset("data/test/disgust", 1)
fear_test = load_dataset("data/test/fear", 2)
happy_test = load_dataset("data/test/happy", 3)
neutral_test = load_dataset("data/test/neutral", 4)
sad_test = load_dataset("data/test/sad", 5)
surprise_test = load_dataset("data/test/surprise", 6)

In [5]:
# On additionne toutes les images ensemble

# Création du X train
X_train = np.concatenate((angry_train[0], disgust_train[0], fear_train[0],
happy_train[0], neutral_train[0], sad_train[0], surprise_train[0]), axis=0)

# Création du Y train
y_train = np.concatenate((angry_train[1], disgust_train[1], fear_train[1],
happy_train[1], neutral_train[1], sad_train[1], surprise_train[1]), axis=0)

# Création du X test
X_test = np.concatenate((angry_test[0], disgust_test[0], fear_test[0],
happy_test[0], neutral_test[0], sad_test[0], surprise_test[0]), axis=0)

# Création du Y test
y_test = np.concatenate((angry_test[1], disgust_test[1], fear_test[1],
happy_test[1], neutral_test[1], sad_test[1], surprise_test[1]), axis=0)

In [6]:
# Création du modèle Tensorflow
model = tf.keras.models.Sequential()

In [7]:
# Ajout des différentes couches de neurones à notre modèle
model.add(tf.keras.layers.Conv2D(64, 7, padding="same", activation='relu', input_shape=(48,48,1)))

model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'))

model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(128, 3, padding="same", activation='relu'))

model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(7, activation='softmax'))

In [8]:
# Compilation du modèle
model.compile(optimizer=tf.keras.optimizers.Adamax(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/100
3589/3589 [==============================] - 58s 16ms/step - loss: 1.8474 - accuracy: 0.2294 - val_loss: 1.7983 - val_accuracy: 0.2471
Epoch 2/100
3589/3589 [==============================] - 58s 16ms/step - loss: 1.7677 - accuracy: 0.2694 - val_loss: 1.6733 - val_accuracy: 0.3607
Epoch 3/100
3589/3589 [==============================] - 58s 16ms/step - loss: 1.6743 - accuracy: 0.3340 - val_loss: 1.6001 - val_accuracy: 0.3902
Epoch 4/100
3589/3589 [==============================] - 56s 15ms/step - loss: 1.6327 - accuracy: 0.3606 - val_loss: 1.5494 - val_accuracy: 0.3983
Epoch 5/100
3589/3589 [==============================] - 56s 15ms/step - loss: 1.5915 - accuracy: 0.3762 - val_loss: 1.5134 - val_accuracy: 0.4120
Epoch 6/100
3589/3589 [==============================] - 56s 16ms/step - loss: 1.5744 - accuracy: 0.3887 - val_loss: 1.4859 - val_accuracy: 0.4244
Epoch 7/100
3589/3589 [==============================] - 57s 16ms/step - loss: 1.5378 - accuracy: 0.4088 - val_loss: 1

In [10]:
model.save("emotion_model")

INFO:tensorflow:Assets written to: emotion_model/assets
